https://deap.readthedocs.io/en/master/examples/gp_parity.html
https://github.com/DEAP/deap/blob/4db155fb3c4fe1678f8d7cd03a638248a1a2f447/examples/gp/parity.py

In [1]:
import random
import operator

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [6]:
# Initialize Parity problem input and output matrices
PARITY_FANIN_M = 6
PARITY_SIZE_M = 2**PARITY_FANIN_M

inputs = [None] * PARITY_SIZE_M
outputs = [None] * PARITY_SIZE_M

for i in range(PARITY_SIZE_M):
    inputs[i] = [None] * PARITY_FANIN_M
    value = i
    dividor = PARITY_SIZE_M
    parity = 1
    for j in range(PARITY_FANIN_M):
        dividor /= 2
        if value >= dividor:
            inputs[i][j] = 1
            parity = int(not parity)
            value -= dividor
        else:
            inputs[i][j] = 0
    outputs[i] = parity

inputs, outputs

([[0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1, 1],
  [0, 0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0, 1],
  [0, 0, 0, 1, 1, 0],
  [0, 0, 0, 1, 1, 1],
  [0, 0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0, 1],
  [0, 0, 1, 0, 1, 0],
  [0, 0, 1, 0, 1, 1],
  [0, 0, 1, 1, 0, 0],
  [0, 0, 1, 1, 0, 1],
  [0, 0, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1],
  [0, 1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 1],
  [0, 1, 0, 0, 1, 0],
  [0, 1, 0, 0, 1, 1],
  [0, 1, 0, 1, 0, 0],
  [0, 1, 0, 1, 0, 1],
  [0, 1, 0, 1, 1, 0],
  [0, 1, 0, 1, 1, 1],
  [0, 1, 1, 0, 0, 0],
  [0, 1, 1, 0, 0, 1],
  [0, 1, 1, 0, 1, 0],
  [0, 1, 1, 0, 1, 1],
  [0, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 0, 1],
  [0, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1],
  [1, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 1],
  [1, 0, 0, 0, 1, 0],
  [1, 0, 0, 0, 1, 1],
  [1, 0, 0, 1, 0, 0],
  [1, 0, 0, 1, 0, 1],
  [1, 0, 0, 1, 1, 0],
  [1, 0, 0, 1, 1, 1],
  [1, 0, 1, 0, 0, 0],
  [1, 0, 1, 0, 0, 1],
  [1, 0, 1, 0, 1, 0],
  [1, 0, 1, 0, 1, 1],
  [1, 0, 1, 1, 0, 0],
  [1, 0, 1

In [27]:
for i, j in zip(inputs, outputs):
    print(i, j)
# 1の数が偶数なら1,奇数なら0を出す

[0, 0, 0, 0, 0, 0] 1
[0, 0, 0, 0, 0, 1] 0
[0, 0, 0, 0, 1, 0] 0
[0, 0, 0, 0, 1, 1] 1
[0, 0, 0, 1, 0, 0] 0
[0, 0, 0, 1, 0, 1] 1
[0, 0, 0, 1, 1, 0] 1
[0, 0, 0, 1, 1, 1] 0
[0, 0, 1, 0, 0, 0] 0
[0, 0, 1, 0, 0, 1] 1
[0, 0, 1, 0, 1, 0] 1
[0, 0, 1, 0, 1, 1] 0
[0, 0, 1, 1, 0, 0] 1
[0, 0, 1, 1, 0, 1] 0
[0, 0, 1, 1, 1, 0] 0
[0, 0, 1, 1, 1, 1] 1
[0, 1, 0, 0, 0, 0] 0
[0, 1, 0, 0, 0, 1] 1
[0, 1, 0, 0, 1, 0] 1
[0, 1, 0, 0, 1, 1] 0
[0, 1, 0, 1, 0, 0] 1
[0, 1, 0, 1, 0, 1] 0
[0, 1, 0, 1, 1, 0] 0
[0, 1, 0, 1, 1, 1] 1
[0, 1, 1, 0, 0, 0] 1
[0, 1, 1, 0, 0, 1] 0
[0, 1, 1, 0, 1, 0] 0
[0, 1, 1, 0, 1, 1] 1
[0, 1, 1, 1, 0, 0] 0
[0, 1, 1, 1, 0, 1] 1
[0, 1, 1, 1, 1, 0] 1
[0, 1, 1, 1, 1, 1] 0
[1, 0, 0, 0, 0, 0] 0
[1, 0, 0, 0, 0, 1] 1
[1, 0, 0, 0, 1, 0] 1
[1, 0, 0, 0, 1, 1] 0
[1, 0, 0, 1, 0, 0] 1
[1, 0, 0, 1, 0, 1] 0
[1, 0, 0, 1, 1, 0] 0
[1, 0, 0, 1, 1, 1] 1
[1, 0, 1, 0, 0, 0] 1
[1, 0, 1, 0, 0, 1] 0
[1, 0, 1, 0, 1, 0] 0
[1, 0, 1, 0, 1, 1] 1
[1, 0, 1, 1, 0, 0] 0
[1, 0, 1, 1, 0, 1] 1
[1, 0, 1, 1, 1, 0] 1
[1, 0, 1, 1, 

In [29]:
len(outputs)

64

In [11]:
pset = gp.PrimitiveSet("MAIN", PARITY_FANIN_M, "IN") # PARITY_FANIN_M==6 は引数の数
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.xor, 2)
pset.addPrimitive(operator.not_, 1)
pset.addTerminal(1)
pset.addTerminal(0)

In [12]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

In [13]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=3, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [15]:
def evalParity(individual):
    func = toolbox.compile(expr=individual)
    return sum(func(*in_) == out for in_, out in zip(inputs, outputs)),

toolbox.register("evaluate", evalParity)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [25]:
def main():
    random.seed(21)
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats, halloffame=hof)
    
    return pop, stats, hof

In [26]:
pop, stats, hof = main()

gen	nevals	avg    	std    	min	max
0  	300   	31.9867	1.23551	24 	48 
1  	185   	31.9933	0.565646	24 	36 
2  	175   	32.0633	0.415518	30 	36 
3  	184   	32.1567	0.65227 	30 	37 
4  	182   	32.25  	0.852936	30 	37 
5  	194   	32.4667	1.20922 	28 	37 
6  	183   	32.91  	1.62129 	28 	38 
7  	199   	33.3   	1.84842 	28 	40 
8  	175   	33.7433	2.03735 	28 	40 
9  	183   	34.2033	2.17148 	28 	40 
10 	182   	34.3933	2.27126 	28 	40 
11 	192   	34.6667	2.34852 	22 	40 
12 	173   	34.9267	2.4889  	28 	40 
13 	165   	35.1533	2.70736 	24 	42 
14 	164   	35.51  	2.72578 	26 	42 
15 	174   	35.7467	3.00153 	24 	44 
16 	166   	36.06  	3.30501 	28 	44 
17 	179   	36.3633	3.84075 	20 	44 
18 	183   	37.01  	3.96357 	20 	44 
19 	181   	37.87  	3.79558 	26 	44 
20 	183   	38.1267	4.41633 	24 	44 
21 	185   	38.57  	4.43679 	24 	46 
22 	201   	38.5933	5.01942 	18 	46 
23 	165   	39.67  	4.82159 	21 	48 
24 	178   	40.65  	4.8943  	20 	48 
25 	177   	41.0133	5.06029 	25 	48 
26 	158   	41.51  	5.65537 	18